In [124]:
from os.path import join as pj
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
import nltk
nltk.download('wordnet')
from sklearn.model_selection import train_test_split
np.random.seed(400)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Data with english abstract and published in 2020

In [15]:
path = 'data/'
file_name = 'metadata_complet_with_abstract.csv'
df = pd.read_csv(pj(path, file_name))
df_en = df.copy()
df_en = df[df['language']=='en']
df_en_20 = df_en[df_en.publish_time.str[:4]=='2020']

Deal with duplicates

In [12]:
df_en_20 = df_en_20.drop_duplicates(['title','abstract'])

In [14]:
df_en_20.shape

(2702, 18)

Train on abstract

In [65]:
train, test = train_test_split(df_en_20.abstract, test_size=0.1, random_state=42)

In [89]:
def lemmatize_stemming(text):
    return PorterStemmer().stem(WordNetLemmatizer().lemmatize(text, pos='v'))
# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

processed_docs=[]
for t in df_en_20.abstract:
    processed_docs.append(preprocess(t))
    
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 8, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [90]:
lda_model.save('lda_2.model')

Results

In [91]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic {}: \nWords: {}".format(idx, topic ))
    print("\n")

Topic 0: 
Words: 0.015*"chest" + 0.015*"lung" + 0.013*"imag" + 0.011*"lesion" + 0.010*"swab" + 0.010*"acid" + 0.010*"nucleic" + 0.010*"neg" + 0.010*"famili" + 0.009*"incub"


Topic 1: 
Words: 0.012*"critic" + 0.011*"admiss" + 0.011*"count" + 0.011*"blood" + 0.010*"significantli" + 0.010*"outcom" + 0.010*"mortal" + 0.009*"lymphocyt" + 0.009*"injuri" + 0.008*"common"


Topic 2: 
Words: 0.009*"travel" + 0.007*"chines" + 0.007*"contact" + 0.006*"person" + 0.006*"pandem" + 0.006*"commun" + 0.006*"individu" + 0.006*"manag" + 0.006*"intern" + 0.006*"nation"


Topic 3: 
Words: 0.019*"genom" + 0.016*"assay" + 0.014*"influenza" + 0.012*"strain" + 0.010*"sensit" + 0.009*"diagnost" + 0.009*"anim" + 0.008*"mer" + 0.007*"type" + 0.007*"gene"


Topic 4: 
Words: 0.019*"genom" + 0.014*"gene" + 0.012*"strain" + 0.009*"genet" + 0.008*"mutat" + 0.007*"pangolin" + 0.007*"transcript" + 0.007*"select" + 0.006*"singl" + 0.006*"reveal"


Topic 5: 
Words: 0.019*"vaccin" + 0.019*"bind" + 0.018*"structur" + 0.017

Model 2 :
-Topic 0: chest lung images 'nucleic' 

-Topic 1: patients, evolution de la maladie

-Topic 2: nation, contact, travel, pandemie

-Topic 3: gene genome autres maladies (grippe) diagnostic

-Topic 4: genetique

-Topic 5: vaccin

-Topic 6: Hubei

-Topic 7: medicaments

Model 1
-Topic 0: Hubei, la quarantaine, contamination par le voyage (fermeture des frontières), prediction, éviter le contact social 

-Topic 1: Mortalité, Femmes enceintes dangerosité ? et enfants, asymptomatiques ?

-Topic 2: Chest and Lung Scans, lesions on images, opac => traces sur les images (grind ??)

-Topic 3: Nombres de cellules cytokin, lymphocyt dans le sang, blessure 

-Topic 4: vaccin, gene, genome etc.

-Topic 5: Tests medicaments

-Topic 6: Autres maladies, personnes à risque, diagnostic

-Topic 7: Corps médical et management de la crise 

Test

In [75]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(test[37380]))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic {}: {}".format(score, index, lda_model.print_topic(-1)))

Score: 0.9831529855728149	 Topic 0: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"


In [74]:
test[37380]

'Abstract Only a month after the outbreak of pneumonia caused by 2019-nCoV, more than forty-thousand people were infected. This put enormous pressure on the Chinese government, medical healthcare provider, and the general public, but also made the international community deeply nervous. On the 25th day after the outbreak, the Chinese government implemented strict traffic restrictions on the area where the 2019-nCoV had originated—Hubei province, whose capital city is Wuhan. Ten days later, the rate of increase of cases in Hubei showed a significant difference (p = 0.0001) compared with the total rate of increase in other provinces of China. These preliminary data suggest the effectiveness of a traffic restriction policy for this pandemic thus far. At the same time, solid financial support and improved research ability, along with network communication technology, also greatly facilitated the application of epidemic prevention measures. These measures were motivated by the need to provi

In [73]:
test

37380    Abstract Only a month after the outbreak of pn...
36175    While the coronavirus death rate may be lower ...
570      An outbreak of novel betacoronavirus, SARS-CoV...
380      Objective: To investigate the correlation betw...
1788     As of 28 February 2020, Italy had 888 cases of...
1469     In the recent issues of IRGEE, we have been as...
26151    Abstract The National Bioforensic Analysis Cen...
520      AbstractA new coronavirus SARS-CoV-2, recently...
1991     Objective To understand the possible transmiss...
402      AbstractThe 2013-2016 West Africa EBOV epidemi...
37167    A novel bat-origin coronavirus emerged in Wuha...
164      Cases from the ongoing outbreak of atypical pn...
1597     As the world is witnessing the epidemic of COV...
34317    Abstract Importation and transmission of measl...
452      SUMMARYIn the analysis of genomic sequence dat...
1924     Objective The purpose of this review of COVID-...
734      Background.This research aims to analyze the c.

In [76]:
test[570]

'An outbreak of novel betacoronavirus, SARS-CoV-2 (formerly named 2019-nCoV), began in Wuhan, China in December 2019 and the COVID-19 disease associated with infection has since spread rapidly to multiple countries. Here we report the development of SARS-CoV-2 DETECTR, a rapid (~30 min), low-cost, and accurate CRISPR-Cas12 based lateral flow assay for detection of SARS-CoV-2. We validated this method using contrived reference samples and clinical samples from infected US patients and demonstrated comparable performance to the US CDC SARS-CoV-2 real-time RT-PCR assay.'

In [77]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(test[570]))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic {}: {}".format(score, index, lda_model.print_topic(-1)))

Score: 0.5852038860321045	 Topic 6: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"
Score: 0.19930173456668854	 Topic 4: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"
Score: 0.1786642223596573	 Topic 0: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"


In [78]:
test[380]

'Objective: To investigate the correlation between clinical characteristics and cardiac injury of COVID-2019 pneumonia. Methods: In this retrospective, single-center study, 41 consecutive corona virus disease 2019 (COVID-2019) patients (including 2 deaths) of COVID-2019 in Beijing Youan Hospital, China Jan 21 to Feb 03, 2020, were involved in this study. The high risk factors of cardiac injury in different COVID-2019 patients were analyzed. Computed tomographic (CT) imaging of epicardial adipose tissue (EAT) has been used to demonstrate the cardiac inflammation of COVID-2019. Results：Of the 41 COVID-2019 patients, 2 (4.88%), 32 (78.05%), 4 (9.75%) and 3 (7.32%) patients were clinically diagnosed as light, mild, severe and critical cases, according to the 6th guidance issued by the National Health Commission of China. 10 (24.4%) patients had underlying complications, such as hypertension, CAD, type 2 diabetes mellites and tumor. The peak value of TnI in critical patients is 40-fold more

In [79]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(test[380]))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic {}: {}".format(score, index, lda_model.print_topic(-1)))

Score: 0.8044670820236206	 Topic 3: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"
Score: 0.1229139044880867	 Topic 0: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"
Score: 0.06313582509756088	 Topic 2: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"


In [86]:
train[2041]

"The new coronavirus pneumonia has been listed as one of the Class B infectious disease but is managed as Class A infectious disease. To prevent and control its spread in hospitals, the outpatient department is the first key gate. Based on the relevant diagnosis and treatment strategies of the National Health Commission of the People's Republic of China, combined with the actual situation of the hospital's epidemic prevention and control work, this article formulated comprehensive prevention and control strategies from the perspective of the patients and staffs. From the aspects of organization and leadership, medical epidemic prevention, pre-screening and screening, process formulation, admission management, cleaning and disinfection, epidemic report, prevention and control supervision, personnel and material deployment, patient education, comprehensive management, personnel management and psychological support and so on, advice and guidance on prevention and control of this infectiou

In [87]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(train[2041]))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic {}: {}".format(score, index, lda_model.print_topic(-1)))

Score: 0.9796350598335266	 Topic 7: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"


Mise en forme des résultats dans le dataframe

In [128]:
nb_topics = 8
for i in range(nb_topics):
    df_en_20['topic_'+str(i)]=None
i=0
idx = df_en_20.index
for a in df_en_20.abstract:
    bow_vector = dictionary.doc2bow(preprocess(a))
    for topic, score in lda_model[bow_vector]:
        df_en_20['topic_'+str(topic)].loc[idx[i]] = score
    i+=1

C:\Users\tr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\tr\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [130]:
df_en_20.to_csv('metadata_en_20_topics.csv', index=False)